In [1]:
import pandas as pd

In [3]:
places_df = pd.read_json("./todo_add.json")

In [ ]:
print(places_df.head())

In [6]:
print(places_df.isnull().sum())

location_id        0
place_name         0
city               0
rating             0
latitude           0
longitude          0
opening_hours      8
contact_number    68
photo             14
address            0
min_price          0
max_price          0
activity           0
category           0
description        0
dtype: int64


In [7]:
places_df['rating'] = places_df['rating'].astype(float)
places_df['latitude'] = places_df['latitude'].astype(float)
places_df['longitude'] = places_df['longitude'].astype(float)

In [8]:
places_df['rating'] = (places_df['rating'] - places_df['rating'].min()) / (places_df['rating'].max() - places_df['rating'].min())
places_df['latitude'] = (places_df['latitude'] - places_df['latitude'].min()) / (places_df['latitude'].max() - places_df['latitude'].min())
places_df['longitude'] = (places_df['longitude'] - places_df['longitude'].min()) / (places_df['longitude'].max() - places_df['longitude'].min())

In [9]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Pilih atribut yang relevan
relevant_attributes = ['activity', 'category', 'rating', 'latitude', 'longitude']
relevant_df = places_df[relevant_attributes]

# Encoding kategori
label_encoder = LabelEncoder()
relevant_df['activity'] = label_encoder.fit_transform(relevant_df['activity'])
relevant_df['category'] = label_encoder.fit_transform(relevant_df['category'])

# Normalisasi atribut numerik
scaler = MinMaxScaler()
relevant_df[['rating', 'latitude', 'longitude']] = scaler.fit_transform(relevant_df[['rating', 'latitude', 'longitude']])

# Menampilkan informasi setelah preprocessing atribut
print(relevant_df.head())

   activity  category  rating  latitude  longitude
0         2         2    0.25  0.173506   0.521923
1         1        13    0.50  0.162914   0.492798
2         2         6    0.50  0.216779   0.492527
3         2         2    0.50  0.159628   0.488951
4         1        13    0.50  0.158223   0.482900


<ipython-input-9-1ba633adb09f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df['activity'] = label_encoder.fit_transform(relevant_df['activity'])
<ipython-input-9-1ba633adb09f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df['category'] = label_encoder.fit_transform(relevant_df['category'])
<ipython-input-9-1ba633adb09f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [10]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Memisahkan atribut dan target
X = relevant_df.drop('rating', axis=1)
y = relevant_df['rating']

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat dan melatih model KNN
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

# Memprediksi rating menggunakan model yang dilatih
y_pred = knn.predict(X_test)

# Evaluasi model menggunakan mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 0.07416666666666667


In [11]:
from sklearn.neighbors import KNeighborsRegressor, BallTree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Memisahkan atribut dan target
X = relevant_df.drop('rating', axis=1)
y = relevant_df['rating']

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat dan melatih model KNN dengan Ball Tree
knn = KNeighborsRegressor(n_neighbors=5, algorithm='ball_tree')
knn.fit(X_train, y_train)

# Memprediksi rating menggunakan model yang dilatih
y_pred = knn.predict(X_test)

# Evaluasi model menggunakan Mean Squared Error (MSE)
mse_knn = mean_squared_error(y_test, y_pred)
print('MSE KNN:', mse_knn)

MSE KNN: 0.07416666666666667


In [14]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Memisahkan atribut dan target
X = relevant_df.drop('rating', axis=1)
y = relevant_df['rating']

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisasi data latih dan data uji
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Membangun model Neural Network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Mengompilasi model
model.compile(optimizer='adam', loss='mse')

# Melatih model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Memprediksi rating menggunakan model yang dilatih
y_pred = model.predict(X_test)

# Evaluasi model menggunakan Mean Squared Error (MSE)
mse_nn = mean_squared_error(y_test, y_pred)
print('MSE Neural Network:', mse_nn)

model.save('neural_network_model.h5')

Epoch 1/10
3/3 [==============================] - 1s 5ms/step - loss: 0.2948
Epoch 2/10
3/3 [==============================] - 0s 5ms/step - loss: 0.2023
Epoch 3/10
3/3 [==============================] - 0s 5ms/step - loss: 0.1375
Epoch 4/10
3/3 [==============================] - 0s 5ms/step - loss: 0.1019
Epoch 5/10
3/3 [==============================] - 0s 5ms/step - loss: 0.0938
Epoch 6/10
3/3 [==============================] - 0s 6ms/step - loss: 0.0960
Epoch 7/10
3/3 [==============================] - 0s 6ms/step - loss: 0.0989
Epoch 8/10
3/3 [==============================] - 0s 6ms/step - loss: 0.0965
Epoch 9/10
3/3 [==============================] - 0s 7ms/step - loss: 0.0907
Epoch 10/10
1/1 [==============================] - 0s 67ms/step
MSE Neural Network: 0.05976963089969268


In [15]:
# Load the saved model
loaded_model = tf.keras.models.load_model('neural_network_model.h5')

In [41]:
import numpy as np

# User input
user_activity = 2
user_category = 5
user_latitude = -1.2654
user_longitude = 116.8311

# Membuat array numpy dari user input
user_input = np.array([[user_activity, user_category, user_latitude, user_longitude]])

# Memprediksi rating menggunakan model yang dimuat
user_rating = loaded_model.predict(user_input)

# Menampilkan hasil prediksi rating
print("Predicted Rating:", user_rating[0][0])

1/1 [==============================] - 0s 28ms/step
Predicted Rating: 30.038074


In [28]:
from sklearn.preprocessing import OneHotEncoder

# Membuat encoder OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)


# Mengubah user input menjadi array numpy
user_input = np.array([[user_activity, user_category, user_latitude, user_longitude]])

# Melakukan encoding pada user input
user_input_encoded = onehot_encoder.fit_transform(user_input)

# Melakukan normalisasi pada user input
user_input_scaled = scaler.transform(user_input_encoded)

# Memprediksi rating menggunakan model yang dimuat
user_rating = loaded_model.predict(user_input_scaled)

# Menampilkan hasil prediksi rating
print("Predicted Rating:", user_rating[0][0])

1/1 [==============================] - 0s 58ms/step
Predicted Rating: 0.44024295


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [42]:
# Memprediksi rating menggunakan model yang dimuat
user_rating = loaded_model.predict(user_input_scaled)

# Mengambil hasil prediksi rating
predicted_rating = user_rating[0][0]

# Mengambil data yang sesuai dengan input pengguna
filtered_data = places_df[(places_df['activity'] == user_activity) & (places_df['category'] == user_category) & (places_df['latitude'] == user_latitude) & (places_df['longitude'] == user_longitude)]

# Mengambil informasi yang relevan dari data yang terfilter
result = filtered_data[['place_name', 'city', 'rating', 'address', 'min_price', 'max_price', 'description']]

# Menambahkan kolom predicted_rating ke dalam hasil
result['predicted_rating'] = predicted_rating

# Mengembalikan hasil sebagai dataframe
print(result)

1/1 [==============================] - 0s 24ms/step
Empty DataFrame
Columns: [place_name, city, rating, address, min_price, max_price, description, predicted_rating]
Index: []
